In [300]:
import pandas as pd
import numpy as np
import os

In [301]:
# Загружаем данные
netflix_data_path = os.path.join(os.getcwd(), 'data', 'netflix_titles.csv')
df = pd.read_csv(netflix_data_path)
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [302]:
def str_to_list(sourse_str) -> list:
    if not isinstance(sourse_str, str):
        return np.nan

    result = []
    splited_str = str(sourse_str).strip().split(',')
    for item in splited_str:
        result.append(item.upper().strip())
    return result

assert str_to_list('www, fff') == ['WWW', 'FFF']
assert str_to_list('www,fff') == ['WWW', 'FFF']
assert str_to_list(' www, fff ') == ['WWW', 'FFF']
assert pd.isna(str_to_list(np.nan))

In [303]:
df.loc[:, 'cast'] = df.loc[:, 'cast'].apply(str_to_list)
df.loc[:, 'director'] = df.loc[:, 'director'].apply(str_to_list)
df.loc[:, 'country'] = df.loc[:, 'country'].apply(str_to_list)
df.loc[:, 'listed_in'] = df.loc[:, 'listed_in'].apply(str_to_list)
df.loc[:, 'date_added'] = pd.to_datetime(df.loc[:, 'date_added'].str.strip())
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,[KIRSTEN JOHNSON],NaN,[UNITED STATES],2021-09-25 00:00:00,2020,PG-13,90 min,[DOCUMENTARIES],"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"[AMA QAMATA, KHOSI NGEMA, GAIL MABALANE, THABA...",[SOUTH AFRICA],2021-09-24 00:00:00,2021,TV-MA,2 Seasons,"[INTERNATIONAL TV SHOWS, TV DRAMAS, TV MYSTERIES]","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,[JULIEN LECLERCQ],"[SAMI BOUAJILA, TRACY GOTOAS, SAMUEL JOUY, NAB...",NaN,2021-09-24 00:00:00,2021,TV-MA,1 Season,"[CRIME TV SHOWS, INTERNATIONAL TV SHOWS, TV AC...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021-09-24 00:00:00,2021,TV-MA,1 Season,"[DOCUSERIES, REALITY TV]","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"[MAYUR MORE, JITENDRA KUMAR, RANJAN RAJ, ALAM ...",[INDIA],2021-09-24 00:00:00,2021,TV-MA,2 Seasons,"[INTERNATIONAL TV SHOWS, ROMANTIC TV SHOWS, TV...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,[DAVID FINCHER],"[MARK RUFFALO, JAKE GYLLENHAAL, ROBERT DOWNEY ...",[UNITED STATES],2019-11-20 00:00:00,2007,R,158 min,"[CULT MOVIES, DRAMAS, THRILLERS]","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,2019-07-01 00:00:00,2018,TV-Y7,2 Seasons,"[KIDS' TV, KOREAN TV SHOWS, TV COMEDIES]","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,[RUBEN FLEISCHER],"[JESSE EISENBERG, WOODY HARRELSON, EMMA STONE,...",[UNITED STATES],2019-11-01 00:00:00,2009,R,88 min,"[COMEDIES, HORROR MOVIES]",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,[PETER HEWITT],"[TIM ALLEN, COURTENEY COX, CHEVY CHASE, KATE M...",[UNITED STATES],2020-01-11 00:00:00,2006,PG,88 min,"[CHILDREN & FAMILY MOVIES, COMEDIES]","Dragged from civilian life, a former superhero..."


In [304]:
def get_int_from_duration(duration: str) -> int:
    if not isinstance(duration, str):
        return np.nan
    
    return int(''.join(filter(str.isdigit, duration)))

assert get_int_from_duration('10 seasons') == 10
assert get_int_from_duration('min 10') == 10
assert get_int_from_duration(' 10 ') == 10
assert get_int_from_duration('10') == 10
assert pd.isna(get_int_from_duration(np.nan))

In [305]:
df.loc[:, 'duration_season'] = df.loc[df['type'] == 'TV Show', 'duration'].apply(get_int_from_duration)
df.loc[:, 'duration_min'] = df.loc[df['type'] == 'Movie', 'duration'].apply(get_int_from_duration)
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration_season,duration_min
0,s1,Movie,Dick Johnson Is Dead,[KIRSTEN JOHNSON],NaN,[UNITED STATES],2021-09-25 00:00:00,2020,PG-13,90 min,[DOCUMENTARIES],"As her father nears the end of his life, filmm...",NaN,90.0
1,s2,TV Show,Blood & Water,NaN,"[AMA QAMATA, KHOSI NGEMA, GAIL MABALANE, THABA...",[SOUTH AFRICA],2021-09-24 00:00:00,2021,TV-MA,2 Seasons,"[INTERNATIONAL TV SHOWS, TV DRAMAS, TV MYSTERIES]","After crossing paths at a party, a Cape Town t...",2.0,NaN
2,s3,TV Show,Ganglands,[JULIEN LECLERCQ],"[SAMI BOUAJILA, TRACY GOTOAS, SAMUEL JOUY, NAB...",NaN,2021-09-24 00:00:00,2021,TV-MA,1 Season,"[CRIME TV SHOWS, INTERNATIONAL TV SHOWS, TV AC...",To protect his family from a powerful drug lor...,1.0,NaN
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021-09-24 00:00:00,2021,TV-MA,1 Season,"[DOCUSERIES, REALITY TV]","Feuds, flirtations and toilet talk go down amo...",1.0,NaN
4,s5,TV Show,Kota Factory,NaN,"[MAYUR MORE, JITENDRA KUMAR, RANJAN RAJ, ALAM ...",[INDIA],2021-09-24 00:00:00,2021,TV-MA,2 Seasons,"[INTERNATIONAL TV SHOWS, ROMANTIC TV SHOWS, TV...",In a city of coaching centers known to train I...,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,[DAVID FINCHER],"[MARK RUFFALO, JAKE GYLLENHAAL, ROBERT DOWNEY ...",[UNITED STATES],2019-11-20 00:00:00,2007,R,158 min,"[CULT MOVIES, DRAMAS, THRILLERS]","A political cartoonist, a crime reporter and a...",NaN,158.0
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,2019-07-01 00:00:00,2018,TV-Y7,2 Seasons,"[KIDS' TV, KOREAN TV SHOWS, TV COMEDIES]","While living alone in a spooky town, a young g...",2.0,NaN
8804,s8805,Movie,Zombieland,[RUBEN FLEISCHER],"[JESSE EISENBERG, WOODY HARRELSON, EMMA STONE,...",[UNITED STATES],2019-11-01 00:00:00,2009,R,88 min,"[COMEDIES, HORROR MOVIES]",Looking to survive in a world taken over by zo...,NaN,88.0
8805,s8806,Movie,Zoom,[PETER HEWITT],"[TIM ALLEN, COURTENEY COX, CHEVY CHASE, KATE M...",[UNITED STATES],2020-01-11 00:00:00,2006,PG,88 min,"[CHILDREN & FAMILY MOVIES, COMEDIES]","Dragged from civilian life, a former superhero...",NaN,88.0


In [306]:
df.loc[:, 'duration min'] = df.loc[df['rating'].isin(['74 min', '84 min', '66 min']), 'rating'].apply(get_int_from_duration)
df.loc[df['rating'].isin(['74 min', '84 min', '66 min']), 'rating'] = np.nan
df.loc[df['rating'].isna(), 'rating'] = 'unknown'

In [307]:
df = df.sort_values(by='date_added').reset_index().drop(columns=['index'])
df['id'] = df.index + 1
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration_season,duration_min,duration min,id
0,s5958,Movie,To and From New York,[SORIN DAN MIHALCESCU],"[BARBARA KING, SHAANA DIYA, JOHN KRISIUKENAS, ...",[UNITED STATES],2008-01-01 00:00:00,2006,TV-MA,81 min,"[DRAMAS, INDEPENDENT MOVIES, THRILLERS]","While covering a story in New York City, a Sea...",NaN,81.0,NaN,1
1,s6612,TV Show,Dinner for Five,NaN,NaN,[UNITED STATES],2008-02-04 00:00:00,2007,TV-MA,1 Season,[STAND-UP COMEDY & TALK SHOWS],"In each episode, four celebrities join host Jo...",1.0,NaN,NaN,2
2,s5957,Movie,Just Another Love Story,[OLE BORNEDAL],"[ANDERS W. BERTHELSEN, REBECKA HEMSE, NIKOLAJ ...",[DENMARK],2009-05-05 00:00:00,2007,TV-MA,104 min,"[DRAMAS, INTERNATIONAL MOVIES]",When he causes a car accident that leaves a yo...,NaN,104.0,NaN,3
3,s5956,Movie,Splatter,[JOE DANTE],"[COREY FELDMAN, TONY TODD, TARA LEIGH, ERIN WA...",[UNITED STATES],2009-11-18 00:00:00,2009,TV-MA,29 min,[HORROR MOVIES],"After committing suicide, a washed-up rocker r...",NaN,29.0,NaN,4
4,s7371,Movie,Mad Ron's Prevues from Hell,[JIM MONACO],"[NICK PAWLOW, JORDU SCHELL, JAY KUSHWARA, MICH...",[UNITED STATES],2010-11-01 00:00:00,1987,NR,84 min,"[CULT MOVIES, HORROR MOVIES]","This collection cherry-picks trailers, forgott...",NaN,84.0,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s7197,TV Show,Kikoriki,NaN,[IGOR DMITRIEV],NaN,NaT,2010,TV-Y,2 Seasons,[KIDS' TV],A wacky rabbit and his gang of animal pals hav...,2.0,NaN,NaN,8803
8803,s7255,TV Show,La Familia P. Luche,NaN,"[EUGENIO DERBEZ, CONSUELO DUVAL, LUIS MANUEL Á...",[UNITED STATES],NaT,2012,TV-14,3 Seasons,"[INTERNATIONAL TV SHOWS, SPANISH-LANGUAGE TV S...","This irreverent sitcom featues Ludovico, Feder...",3.0,NaN,NaN,8804
8804,s7407,TV Show,Maron,NaN,"[MARC MARON, JUDD HIRSCH, JOSH BRENER, NORA ZE...",[UNITED STATES],NaT,2016,TV-MA,4 Seasons,[TV COMEDIES],"Marc Maron stars as Marc Maron, who interviews...",4.0,NaN,NaN,8805
8805,s7848,TV Show,Red vs. Blue,NaN,"[BURNIE BURNS, JASON SALDAÑA, GUSTAVO SOROLA, ...",[UNITED STATES],NaT,2015,NR,13 Seasons,"[TV ACTION & ADVENTURE, TV COMEDIES, TV SCI-FI...","This parody of first-person shooter games, mil...",13.0,NaN,NaN,8806


In [308]:
show = df.loc[:, ['id', 'show_id', 'title', 'type', 'date_added', 'release_year', 'rating', 'description', 'duration_season', 'duration_min']].copy()
show.rename(columns={'show_id': 'source_show_id'}, inplace=True)
show

,id,source_show_id,title,type,date_added,release_year,rating,description,duration_season,duration_min
0,1,s5958,To and From New York,Movie,2008-01-01 00:00:00,2006,TV-MA,"While covering a story in New York City, a Sea...",NaN,81.0
1,2,s6612,Dinner for Five,TV Show,2008-02-04 00:00:00,2007,TV-MA,"In each episode, four celebrities join host Jo...",1.0,NaN
2,3,s5957,Just Another Love Story,Movie,2009-05-05 00:00:00,2007,TV-MA,When he causes a car accident that leaves a yo...,NaN,104.0
3,4,s5956,Splatter,Movie,2009-11-18 00:00:00,2009,TV-MA,"After committing suicide, a washed-up rocker r...",NaN,29.0
4,5,s7371,Mad Ron's Prevues from Hell,Movie,2010-11-01 00:00:00,1987,NR,"This collection cherry-picks trailers, forgott...",NaN,84.0
...,...,...,...,...,...,...,...,...,...,...
8802,8803,s7197,Kikoriki,TV Show,NaT,2010,TV-Y,A wacky rabbit and his gang of animal pals hav...,2.0,NaN
8803,8804,s7255,La Familia P. Luche,TV Show,NaT,2012,TV-14,"This irreverent sitcom featues Ludovico, Feder...",3.0,NaN
8804,8805,s7407,Maron,TV Show,NaT,2016,TV-MA,"Marc Maron stars as Marc Maron, who interviews...",4.0,NaN
8805,8806,s7848,Red vs. Blue,TV Show,NaT,2015,NR,"This parody of first-person shooter games, mil...",13.0,NaN


In [309]:
show_cast = df.dropna(subset=['cast']).explode(column=['cast']).loc[:, ['id', 'cast']].copy().rename(columns={'id': 'show_id'})
show_cast

,show_id,cast
0,1,BARBARA KING
0,1,SHAANA DIYA
0,1,JOHN KRISIUKENAS
0,1,YORIKO HARAGUCHI
0,1,DAVID CRANE
...,...,...
8806,8807,LUKE JUREVICIUS
8806,8807,CRAIG BEHENNA
8806,8807,CHARLOTTE HAMLYN
8806,8807,STAVROULA MOUNTZOURIS


In [310]:
show_director = df.dropna(subset=['director']).explode(column=['director']).loc[:, ['id', 'director']].copy().rename(columns={'id': 'show_id'})
show_director

,show_id,director
0,1,SORIN DAN MIHALCESCU
2,3,OLE BORNEDAL
3,4,JOE DANTE
4,5,JIM MONACO
5,6,ICÍAR BOLLAÍN
...,...,...
8792,8793,JOSÉ LUIS UCHA
8793,8794,HAILE GERIMA
8794,8795,ANDY DEVONSHIRE
8795,8796,THEODORE MELFI


In [311]:
show_country = df.dropna(subset=['country']).explode(column=['country']).loc[:, ['id', 'country']].copy().rename(columns={'id': 'show_id'})
show_country

,show_id,country
0,1,UNITED STATES
1,2,UNITED STATES
2,3,DENMARK
3,4,UNITED STATES
4,5,UNITED STATES
...,...,...
8801,8802,JAPAN
8803,8804,UNITED STATES
8804,8805,UNITED STATES
8805,8806,UNITED STATES


In [312]:
show_listed_in = df.dropna(subset=['listed_in']).explode(column=['listed_in']).loc[:, ['id', 'listed_in']].copy().rename(columns={'id': 'show_id'})
show_listed_in

,show_id,listed_in
0,1,DRAMAS
0,1,INDEPENDENT MOVIES
0,1,THRILLERS
1,2,STAND-UP COMEDY & TALK SHOWS
2,3,DRAMAS
...,...,...
8805,8806,TV ACTION & ADVENTURE
8805,8806,TV COMEDIES
8805,8806,TV SCI-FI & FANTASY
8806,8807,KIDS' TV


In [313]:
import clickhouse_connect

client = clickhouse_connect.get_client(host='###', username='###', password='###', port='###')
client.insert_df(database='NETFLIXDB', df=show, table='show')
client.insert_df(database='NETFLIXDB', df=show_cast, table='show_cast')
client.insert_df(database='NETFLIXDB', df=show_director, table='show_director')
client.insert_df(database='NETFLIXDB', df=show_country, table='show_country')
client.insert_df(database='NETFLIXDB', df=show_listed_in, table='show_listed_in')